In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from transformers import AutoTokenizer,BertPreTrainedModel,BertModel,AdamW
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import get_linear_schedule_with_warmup
from torch.nn import functional as F

In [2]:
data = pd.read_csv('per.csv')
data.head()

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [3]:
dataset = pd.DataFrame(columns=('title_body','category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    dataset.loc[index]= {
        'title_body' : title_body.replace('\n',''),
        'category' : row['Category2'].replace('\n','')
    }

In [4]:
dataset

,title_body,category
0,وزير علوم درجمع استادان نمونه: سن بازنشستگي اس...,آموزشي
1,گردهمايي دانش‌آموختگان موسسه آموزش عالي سوره ب...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ا...,آموزشي
3,همايش يكروزه آسيب شناسي مفهوم روابط عمومي در ب...,اجتماعي
4,وضعيت اقتصادي و ميزان تحصيلات والدين از مهمتري...,آموزشي
...,...,...
10994,/گزارش/ «حقي» حق خود را با طلا گرفت به گزارش ا...,ورزشي
10995,مدير تيم‌هاي ملي كشتي فرنگي: نفرات اول تا سوم ...,ورزشي
10996,تساوي چلسي و ادامه صدرنشيني آرسنال تساوي اشتوت...,ورزشي
10997,شكست ايران از پرتغال در آيينه‌ي خبرگزاري‌ها: د...,ورزشي


In [5]:
le = LabelEncoder()
dataset['category_l'] = le.fit_transform(dataset['category'])
dataset

,title_body,category,category_l
0,وزير علوم درجمع استادان نمونه: سن بازنشستگي اس...,آموزشي,0
1,گردهمايي دانش‌آموختگان موسسه آموزش عالي سوره ب...,آموزشي,0
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ا...,آموزشي,0
3,همايش يكروزه آسيب شناسي مفهوم روابط عمومي در ب...,اجتماعي,1
4,وضعيت اقتصادي و ميزان تحصيلات والدين از مهمتري...,آموزشي,0
...,...,...,...
10994,/گزارش/ «حقي» حق خود را با طلا گرفت به گزارش ا...,ورزشي,10
10995,مدير تيم‌هاي ملي كشتي فرنگي: نفرات اول تا سوم ...,ورزشي,10
10996,تساوي چلسي و ادامه صدرنشيني آرسنال تساوي اشتوت...,ورزشي,10
10997,شكست ايران از پرتغال در آيينه‌ي خبرگزاري‌ها: د...,ورزشي,10


In [6]:
pd.get_dummies(dataset['category'],dtype=float).to_numpy()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [7]:
dataset['category_l']

0         0
1         0
2         0
3         1
4         0
         ..
10994    10
10995    10
10996    10
10997    10
10998    10
Name: category_l, Length: 10999, dtype: int32

In [8]:
train,test = train_test_split(dataset)

In [9]:
train.shape

(8249, 3)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [11]:
model_path_or_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path_or_name)

In [12]:
class BertForMultipleRegression(BertPreTrainedModel):
    def __init__(self,config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        pooled_output = output[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.regressor(pooled_output)
        return logits

In [13]:
model = BertForMultipleRegression.from_pretrained(model_path_or_name)
model = model.to(device)

Some weights of BertForMultipleRegression were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['regressor.weight', 'regressor.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class CustomDataset(Dataset):
    def __init__(self,dataframe,tokenizer,max_length =512):
        self.data=[]
        self.max_length = max_length
        for i, row in dataframe.iterrows():
            text = row['title_body']
            target = row['category_l']
            self.data.append((text,target))
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text,target = self.data[idx]
        input = self.tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        input_ids = input['input_ids'].squeeze(0)
        attention_mask = input['attention_mask'].squeeze(0)
        target = torch.tensor([float(target)])
        return input_ids,attention_mask,target

In [15]:
train_dataset = CustomDataset(train,tokenizer)
test_dataset = CustomDataset(test,tokenizer)

batch_size=8
train_dataloader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [16]:
epochs = 10 
optimizer = AdamW (model.parameters(),
                  lr = 1e-6,
                  eps = 1e-8,
                )

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

C:\ProgramData\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_losses = []
val_losses = []
show_every = 20
epochs = 10
for epoch_i in range(0,epochs):
    store_train_loss = []
    store_val_loss = []
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    total_loss = 0
    model.train()
    for step , batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        model.zero_grad()
        outputs = model(input_ids=input_ids,attention_mask=attention_mask)
        print(outputs)
        loss = F.mse_loss(outputs,labels)
        print(loss)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        store_train_loss.append(loss.item())


======== Epoch 1 / 10 ========
Training...
tensor([[ 0.3351],
        [ 0.1298],
        [ 0.1774],
        [ 0.0183],
        [ 0.1055],
        [ 0.0357],
        [ 0.2168],
        [-0.0731]], grad_fn=<AddmmBackward0>)
[[[4.]
  [6.]
  [5.]
  [6.]
  [5.]
  [7.]
  [4.]
  [3.]]]
tensor(25.3928, grad_fn=<MseLossBackward0>)


In [ ]:
train_dataloader

In [ ]:
dataset